# Vegetated canopies (RAMI-V)

This example loads a pre-defined canopy chosen in the RAMI-V scenario list and
generates an RGB render of it. It demonstrates the usage of the RAMI canopy
loader interface and provides a simple framework to experiments with sensors and
illumination.

We start with a few imports:

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import eradiate
from eradiate.experiments import CanopyAtmosphereExperiment
from eradiate.scenes.biosphere import load_rami_scenario
from eradiate.units import unit_registry as ureg
from eradiate.tutorials import plot_polarfilm

We select the monochromatic mode and assign the wavelengths we select for the RGB channels in a constant variable. We also set the location where we want the RAMI canopy definitions to be stored.

In [ ]:
eradiate.plot.set_style()
plt.style.use("eradiate.mplstyle")

eradiate.set_mode("mono")
RGB = [440.0, 550.0, 660.0] * ureg.nm
unpack_folder = Path(
    "../data/rami_scenarios/"
)  # location where to store canopy definitions
eradiate.fresolver.append("../data")

Next, we perform some basic setup. The selected RAMI scenario is designated by its identifier. We will make an image with a perspective camera, and the default setup presented here places it inside or just above the canopy, depending on the scenario.

In [ ]:
scenario_name = "HET14_WCO_UND"  # Wellington Citrus Orchard (low memory usage)

# Common setup
padding = 3  # apply padding to clone the canopy
atmosphere = {"type": "molecular"}  # basic atmosphere
directional = {"type": "directional", "zenith": 37.0, "azimuth": 248.0}
srf = {"type": "delta", "wavelengths": RGB}
spp = 16  # low value to get results quickly even with a small computer


# Scene-specific setup
def get_camera(scenario_name, film_resolution=(400, 200)):
    if scenario_name == "HET14_WCO_UND":
        # Wellington citrus orchard
        origin = [2, -5, 5.5] * ureg.m
        target = origin + [0.2, 1, -0.05] * ureg.m
    elif scenario_name == "HET50_SAV_PRE":
        # Savanna
        origin = [-5, -3, 2.5] * ureg.m
        target = origin + [0, 1, 0] * ureg.m
    elif scenario_name == "HET51_WWO_TLS":
        # Wytham Wood
        origin = [2, -5, 22.5] * ureg.m
        target = origin + [0.2, 1, -0.05] * ureg.m
    else:
        raise ValueError(f"no parameters for {scenario_name}")

    return {
        "type": "perspective",
        "film_resolution": film_resolution,
        "origin": origin,
        "target": target,
        "up": [0, 0, 1],
        "srf": srf,
    }


camera = get_camera(scenario_name)

Now, let's load the scenario. The definition can be readily passed to the `CanopyAtmosphereExperiment` class. Depending on the scene and hardware, scene loading can take from a few seconds to several minutes.

In [ ]:
scenario_data = load_rami_scenario(
    scenario_name=scenario_name, unpack_folder=unpack_folder, padding=padding
)

exp = CanopyAtmosphereExperiment(
    **scenario_data,
    # atmosphere=None,
    atmosphere=atmosphere,
    measures=camera,
    illumination=directional,
)

exp.init()

Now, let's render the image:

In [ ]:
result = eradiate.run(exp, spp=16)

We can use Eradiate's basic tone mapping function to post-process the image:

In [ ]:
img = (
    eradiate.xarray.interp.dataarray_to_rgb(
        result["radiance"].squeeze(),
        channels=[("w", 660), ("w", 550), ("w", 440)],
        normalize=False,
    )
    * 1.8
)
plt.imshow(img)
plt.axis("off")
# plt.imsave("wco_rami.png", img.clip(0, 1))
plt.show()
plt.close()

## Computing reflectances

Building on this setup, we can compute the top-of-atmosphere and top-of-canopy reflectance for this canopy (at 550 nm). We need to set up a distant radiance sensor that samples the entire horizontal extent of the canopy unit cell:

In [ ]:
# Sensor target definition
target = {
    "type": "rectangle",
    "xmin": -54.1 * ureg.m,
    "xmax": 54.1 * ureg.m,
    "ymin": -51.95 * ureg.m,
    "ymax": 51.95 * ureg.m,
    "z": 12.4 * ureg.m,
}

# Let's visualize the reflected radiance in the entire hemisphere
toa_brf_hemisphere = {
    "type": "hdistant",
    "id": "toa_brf_hemisphere",
    "target": target,
}

# and in the principal plane
toa_brf_pplane = {
    "type": "mdistant",
    "id": "toa_brf_pplane",
    "construct": "hplane",
    "azimuth": directional["azimuth"],
    "zeniths": np.arange(-75, 76, 1),
    "target": target,
}

# We duplicate those sensors for the atmosphere-free setup (we only change their IDs)
toc_brf_hemisphere = {**toa_brf_hemisphere, "id": "toc_brf_hemisphere"}
toc_brf_pplane = {**toa_brf_pplane, "id": "toc_brf_pplane"}

# We'll store all results in a dictionary
result = {}

In [ ]:
# Load the scenario with the same parameters as before, and some padding (5 may
# be insufficient to converge to a quasi-periodic solution, but we use this
# value to keep the computational time reasonable)
scenario_data = load_rami_scenario(
    scenario_name=scenario_name, unpack_folder=unpack_folder, padding=5
)

In [ ]:
# Compute the TOA BRF
exp = CanopyAtmosphereExperiment(
    **scenario_data,
    atmosphere=atmosphere,
    measures=[toa_brf_pplane, toa_brf_hemisphere],
    illumination=directional,
)

exp.init()
result.update(eradiate.run(exp, spp=int(1e5)))

In [ ]:
# Compute the TOC BRF
exp = CanopyAtmosphereExperiment(
    **scenario_data,
    atmosphere=None,
    measures=[toc_brf_pplane, toc_brf_hemisphere],
    illumination=directional,
)

exp.init()
result.update(eradiate.run(exp, spp=int(1e5)))

In [ ]:
# Plot reflectance in polar coordinates
for var, label in [
    ("toa_brf_hemisphere", "TOA BRF"),
    ("toc_brf_hemisphere", "TOC BRF"),
]:
    fig = plot_polarfilm(
        result[var]["brf"].squeeze(),
        theta_max=75,
    )
    ax = fig.axes[1]
    ax.set_title(label)
    ax.scatter(
        np.deg2rad(result[var]["saa"]),
        result[var]["sza"],
        marker="*",
        c="yellow",
        edgecolors="black",
        s=150,
    )
    plt.savefig(f"wco_{var}.pdf", bbox_inches="tight")
    plt.show(fig)

In [ ]:
# Plot reflectance in principal plane
fig, ax = plt.subplots(1, 1, figsize=(6, 3))

for var, label, color in [
    ("toa_brf_pplane", "TOA BRF", "tab:blue"),
    ("toc_brf_pplane", "TOC BRF", "tab:orange"),
]:
    da = result[var]["brf"].squeeze()
    ax.plot(da["vza"].values, da.values, label=label, color=color)

ax.set_xlabel("Viewing Zenith Angle [°]")
ax.legend()
plt.savefig("wco_reflectance_pplane.pdf", bbox_inches="tight")
plt.show()
plt.close()

## Using customized spectral properties

The render we produced is arguably not very realistic: it interpolates the RAMI-V spectral information, which does not particularly well align with the selected wavelengths. We can assign more plausible values when we use the canopy loader.

What we are about to do works for the Wellington citrus orchard scene only: the other scenes will require a different setup. Let's start by importing some data:

In [ ]:
bilambertian_leaf_cellulose = eradiate.fresolver.load_dataset(
    "bsdf/bilambertian_leaf_cellulose.nc"
)
bilambertian_leaf_cellulose = bilambertian_leaf_cellulose.assign_coords(
    {"w": (bilambertian_leaf_cellulose["w"] * 1000.0)}
).assign_attrs({"w": "nm"})

lambertian_soil = eradiate.fresolver.load_dataset("bsdf/lambertian_soil.nc").sel(
    brightness="bright"
)
lambertian_soil = lambertian_soil.assign_coords(
    {"w": (lambertian_soil["w"] * 1000.0)}
).assign_attrs({"w": "nm"})


lambertian_wood = eradiate.fresolver.load_dataset("bsdf/lambertian_disney_2011.nc")[
    "bark"
]
lambertian_wood["w"].attrs["units"] = "nm"

bilambertian_leaf_cellulose["reflectance"].plot(label="leaf_reflectance")
bilambertian_leaf_cellulose["transmittance"].plot(label="leaf_transmittance")
lambertian_soil["reflectance"].plot(label="soil_reflectance")
lambertian_wood.plot(label="wood_reflectance")
plt.title("")
plt.ylabel("")
plt.legend()

We define optical property spectra for both materials. To keep things minimal, we interpolate the spectra on the pre-defined RGB channels; but we could keep the full spectra (clipping NaN values), and it would also work just fine.

In [ ]:
materials = {
    "leaf": {
        "reflectance": {
            "type": "interpolated",
            "wavelengths": RGB,
            "values": bilambertian_leaf_cellulose["reflectance"]
            .dropna("w")
            .interp(w=np.sort(RGB).m, kwargs={"fill_value": "extrapolate"})
            .values,
        },
        "transmittance": {
            "type": "interpolated",
            "wavelengths": RGB,
            "values": bilambertian_leaf_cellulose["transmittance"]
            .dropna("w")
            .interp(w=np.sort(RGB).m, kwargs={"fill_value": "extrapolate"})
            .values,
        },
    },
    "bright_soil": {
        "reflectance": {
            "type": "interpolated",
            "wavelengths": RGB,
            "values": lambertian_soil["reflectance"]
            .dropna("w")
            .interp(w=np.sort(RGB).m, kwargs={"fill_value": "extrapolate"})
            .values,
        }
    },
    "wood": {
        "reflectance": {
            "type": "interpolated",
            "wavelengths": RGB,
            "values": lambertian_wood.dropna("w")
            .interp(w=np.sort(RGB).m, kwargs={"fill_value": "extrapolate"})
            .values,
        }
    },
}

Next, we assign the spectra to each part of the scene:

In [ ]:
trees = [f"CISI{i}" for i in range(1, 11)]
parts = ["Leaves", "Wood"]

mat_to_entry = {
    "wood": sorted(
        [f"{tree}.{part}" for tree in trees for part in parts if part == "Wood"]
    ),
    "leaf": sorted(
        [f"{tree}.{part}" for tree in trees for part in parts if part == "Leaves"]
    ),
}

spectral_data = {"ground": materials["bright_soil"].copy()}
for mat, entries in mat_to_entry.items():
    for entry in entries:
        tree, component = entry.split(".")

        if tree not in spectral_data:
            spectral_data[tree] = {}

        spectral_data[tree][component] = materials[mat].copy()

# spectral_data

Now, we can render the scene against and check that the colours seem more natural:

In [ ]:
# Load the scene with the updated material definitions
scenario_data = load_rami_scenario(
    scenario_name,
    unpack_folder=unpack_folder,
    padding=padding,
    spectral_data=spectral_data,
)

In [ ]:
# Load the Eradiate experiment
exp = CanopyAtmosphereExperiment(
    **scenario_data,
    # atmosphere=None,
    atmosphere=atmosphere,
    measures=camera,
    # measures=get_camera(scenario_name, film_resolution=(1500, 750)),
    illumination=directional,
)

exp.init()

In [ ]:
result = eradiate.run(exp, spp=spp)

In [ ]:
img = (
    eradiate.xarray.interp.dataarray_to_rgb(
        result["radiance"].squeeze(),
        channels=[("w", w) for w in RGB.m[::-1]],
        normalize=False,
    )
    * 1.8
)
plt.imshow(img)
plt.axis("off")
plt.imsave("wco_natural.png", img.clip(0, 1))
plt.show()
plt.close()

And we can also visualize the unit cell from above:

In [ ]:
# Load the Eradiate experiment
scenario_data = load_rami_scenario(
    scenario_name,
    unpack_folder=unpack_folder,
    padding=0,
    spectral_data=spectral_data,
)

exp = CanopyAtmosphereExperiment(
    **scenario_data,
    atmosphere=None,
    measures={
        "type": "perspective",
        "film_resolution": (256, 256),
        "srf": srf,
        "origin": [0, 0, 1240],
        "target": [0, 0, 0],
        "up": [0, 1, 0],
        "fov": 5,
    },
    illumination=directional,
)

exp.init()
result = eradiate.run(exp, spp=16)

img = (
    eradiate.xarray.interp.dataarray_to_rgb(
        result["radiance"].squeeze(),
        channels=[("w", w) for w in RGB.m[::-1]],
        normalize=False,
    )
    * 1.8
)
plt.imshow(img)
plt.axis("off")
plt.imsave("wco_unit_cell.png", img.clip(0, 1))
plt.show()
plt.close()

## Savanna pre-fire setup

The Savanna pre-fire scene definition is heavier than the Wellington citrus orchard, and it has a different layout. In the following, we apply the same approach to load the scene and change its spectra properties to get a setup that produces images similar to what is in the Eradiate v1.0.0 paper.

In [ ]:
scenario_name = "HET50_SAV_PRE"  # Savanna pre-fire (high memory usage)
padding = 1

In [ ]:
reflectance = eradiate.fresolver.load_dataset("bsdf/lambertian_disney_2011.nc")

materials = {
    var: {
        "type": "interpolated",
        "wavelengths": RGB,
        "values": reflectance[var]
        .dropna("w")
        .interp(w=np.sort(RGB).m, kwargs={"fill_value": "extrapolate"})
        .values,
    }
    for var in reflectance.data_vars
}

# materials

In [ ]:
trees = (
    [f"combretum_leafon{x}{flat}" for x in [1, 2] for flat in ["", "_flat"]]
    + [f"combretum_leafoff{x}{flat}" for x in [1, 2, 3, 4, 5] for flat in ["", "_flat"]]
    + [f"merula{x}" for x in [1, 2, 3]]
)
parts = ["Bough", "Leaf1", "measured_branch", "measured_trunk"]

mat_to_entry = {
    "bark": sorted(
        [f"{tree}.{part}" for tree in trees for part in parts if (not part == "Leaf1")]
    ),
    "bright_soil": [],
    "burnt_bark": [],
    "char": [],
    "dry_grass": [f"grass{x}.measured_stem" for x in range(10)],
    "leaf": sorted(
        [
            f"{tree}.{part}"
            for tree in trees
            for part in parts
            if (part == "Leaf1") and (("leafoff" not in tree) or ("leafoff1" in tree))
        ]
    ),
}

spectral_data = {
    "ground": {"reflectance": materials["bright_soil"]},
}
for mat, entries in mat_to_entry.items():
    for entry in entries:
        tree, component = entry.split(".")

        if tree not in spectral_data:
            spectral_data[tree] = {}

        spectral_data[tree][component] = {
            "reflectance": materials[mat],
            "transmittance": 0.0,  # add transmittance value so material is two-sided
        }

# spectral_data

In [ ]:
scenario_data = load_rami_scenario(
    scenario_name,
    unpack_folder=unpack_folder,
    padding=padding,
    spectral_data=spectral_data,
)
# display(scenario_data["canopy"])

# Filter out flat trees (which we don't really see because of the high grass)
to_keep = [
    i
    for i, element in enumerate(scenario_data["canopy"]["instanced_canopy_elements"])
    if "grass" in element["canopy_element"]["id"]
    or (
        (
            "merula" in element["canopy_element"]["id"]
            or "combretum" in element["canopy_element"]["id"]
        )
        and ("flat" not in element["canopy_element"]["id"])
    )
]
scenario_data["canopy"]["instanced_canopy_elements"] = [
    scenario_data["canopy"]["instanced_canopy_elements"][i] for i in to_keep
]

In [ ]:
camera = get_camera(scenario_name)

exp = CanopyAtmosphereExperiment(
    **scenario_data,
    # atmosphere=None,
    atmosphere=atmosphere,
    measures=camera,
    illumination=directional,
)

exp.init()

In [ ]:
result = eradiate.run(exp, spp=spp)

In [ ]:
plt.imshow(
    eradiate.xarray.interp.dataarray_to_rgb(
        result["radiance"].squeeze(),
        channels=[("w", w) for w in RGB.m[::-1]],
        normalize=False,
    )
    * 1.8
)
plt.axis("off")

What's next?

* Adjust simulation parameters (sample count, padding, sensor position, spectral properties, atmosphere).
* Compute the BOA HDRF of the canopy.